In [1]:
import numpy as np

np.set_printoptions(
    suppress=True,
    linewidth=180
)

In [2]:
def get_random_vehicles(num_vehicles, max_vulns):
    import random
    vehicles = [
        {
            "id": i,
            "vulns":[
                {
                    "prob": round(random.uniform(0.1, 0.98),2),
                    "severity": random.randint(1,5),
                }
                for i in range(random.randint(1,max_vulns))
            ]
        }
        for i in range(num_vehicles)
    ]
    for v in vehicles:
        for c in v["vulns"]:
            c["risk"] = round(c["severity"] ** 2 * c["prob"] * 100)/100
    return vehicles

In [3]:
def powerset(s):
    import itertools
    return itertools.chain.from_iterable(itertools.combinations(s, r) for r in range(1, len(s)+1))

In [ ]:
def get_defender_subgame_utility(vehicle_list, attack):
    lookup = set([x["id"] for x in vehicle_list])
    bad = 0
    for v in attack:
        if v["id"] in lookup:
            for c in v["components"]:
                bad += c["prob"] * c["severity"] ** 2
    return len(vehicle_list) - bad

In [ ]:
def get_attacker_utility(platoon, attack):
    lookup = set([x["id"] for x in platoon])
    util = 0
    for v in attack:
        if v["id"] in lookup:
            for c in v["components"]:
                util += c["prob"] * c["severity"] ** 2
    return util

In [ ]:
def generate_utility_matrices(board, attacker_selection_limit):
    possible_platoons = list(powerset(board))
    possible_attacks = [x for x in powerset(board) if len(x) <= attacker_selection_limit]
    defender_utility_matrix = np.zeros((len(possible_attacks),len(possible_platoons)))    
    attacker_utility_matrix = np.zeros((len(possible_platoons),len(possible_attacks)))

    for i, platoon in enumerate(possible_platoons):
        for j, attack in enumerate(possible_attacks):
            defender_utility_matrix[j][i] = get_defender_utility(platoon, attack)
            attacker_utility_matrix[i][j] = get_attacker_utility(platoon, attack)
    
    return defender_utility_matrix, \
        possible_platoons, \
        attacker_utility_matrix, \
        possible_attacks,

In [ ]:
board = generate_board(4,2)
print(board)

[{'id': 0, 'components': [{'prob': 0.67, 'severity': 2, 'risk': 2.68}]}, {'id': 1, 'components': [{'prob': 0.91, 'severity': 5, 'risk': 22.75}]}, {'id': 2, 'components': [{'prob': 0.96, 'severity': 2, 'risk': 3.84}, {'prob': 0.49, 'severity': 1, 'risk': 0.49}]}, {'id': 3, 'components': [{'prob': 0.46, 'severity': 2, 'risk': 1.84}]}]


In [ ]:
def solve(util):
    import scipy.optimize
    
    # print(util.shape)

    c = np.zeros(util.shape[1]+1)
    c[0] = -1

    A_ub = np.ones((util.shape[0], util.shape[1] +1))
    A_ub[:,1:] = util*-1
    b_ub = np.zeros(util.shape[0])

    A_eq = np.ones((1, util.shape[1] + 1))
    A_eq[0][0] = 0
    b_eq = 1

    lb = np.zeros(util.shape[1]+1)
    lb[0] = -10000
    ub = np.ones(util.shape[1]+1)
    ub[0] = 10000
    bounds = np.asarray([lb, ub]).transpose()

    result = scipy.optimize.linprog(
        c=c,
        A_ub=A_ub,
        b_ub=b_ub,
        A_eq=A_eq,
        b_eq=b_eq,
        bounds=bounds,
        # options = {
        #     "tol": 0.001
        #     # "autoscale": True
        # }
        # method="simplex",
        method="highs",
        # method="interior-point",
        # options={"presolve":False},
        # callback = lambda x: zz.append(x.x)
    )
    if result.success:
        print(result.x[0])
        return result.x[1:]
    else:
        print(result)
        raise Exception("Couldn't find solution")

In [ ]:
def_util, def_opt, atk_util, atk_opt = generate_utility_matrices(board, 1)